In [19]:
import os
import csv

DIRECTORIO_IMAGENES = "C:/Users/Jhack Alberth/Downloads/kagglecatsanddogs_5340/PetImages"
archivo_csv = "C:/Users/Jhack Alberth/Downloads/kagglecatsanddogs_5340/etiquetas.csv"

# Crear el archivo CSV
with open(archivo_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["nombre_archivo", "etiqueta"])  # Encabezado

    for categoria in ["Cat", "Dog"]:
        etiqueta = 1 if categoria == "Dog" else 0
        directorio_categoria = os.path.join(DIRECTORIO_IMAGENES, categoria)

        for nombre_archivo in os.listdir(directorio_categoria):
            ruta_completa = os.path.join(directorio_categoria, nombre_archivo)
            if os.path.isfile(ruta_completa):
                writer.writerow([ruta_completa, etiqueta])  # Escribir ruta y etiqueta

In [1]:
!pip install tensorflow_datasets


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importar librerías necesarias
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
import cv2
import os

In [3]:
# Definir rutas y tamaño de las imágenes
TAMANO_IMG = 100
ARCHIVO_CSV = "C:/Users/Jhack Alberth/Downloads/kagglecatsanddogs_5340/etiquetas.csv"

In [4]:
# Leer el archivo CSV
data = pd.read_csv(ARCHIVO_CSV, header=None, names=["ruta", "etiqueta"])

In [5]:
# Función para cargar y preprocesar imágenes
def cargar_y_preprocesar_img(ruta, etiqueta):
    # Cargar la imagen desde la ruta
    img = tf.io.read_file(ruta)
    img = tf.image.decode_jpeg(img, channels=3)  # Decodificar como RGB
    img = tf.image.resize(img, [TAMANO_IMG, TAMANO_IMG])
    img = tf.image.rgb_to_grayscale(img)  # Convertir a escala de grises si lo deseas
    img = img / 255.0  # Normalizar los valores de los píxeles entre 0 y 1
    return img, etiqueta

In [6]:
# Crear el dataset desde el archivo CSV
ruta_dataset = tf.data.Dataset.from_tensor_slices((data["ruta"].values, data["etiqueta"].values))
dataset = ruta_dataset.map(lambda x, y: cargar_y_preprocesar_img(x, y), num_parallel_calls=tf.data.AUTOTUNE)

In [7]:
# Dividir el dataset en entrenamiento y validación (85% - 15%)
datos_entrenamiento = dataset.skip(int(len(dataset) * 0.15))
datos_validacion = dataset.take(int(len(dataset) * 0.15))

In [8]:
# Configurar lotes y prefetch para un entrenamiento eficiente
datos_entrenamiento = datos_entrenamiento.batch(32).prefetch(tf.data.AUTOTUNE)
datos_validacion = datos_validacion.batch(32).prefetch(tf.data.AUTOTUNE)

In [10]:
import pandas as pd
import os

# Cargar el archivo CSV
archivo_csv = "C:/Users/Jhack Alberth/Downloads/kagglecatsanddogs_5340/etiquetas.csv"
data = pd.read_csv(archivo_csv, header=None, names=["ruta", "etiqueta"])

# Filtrar solo las filas donde las rutas existen
data = data[data["ruta"].apply(os.path.exists)]

# Guardar el CSV filtrado para referencias futuras (opcional)
data.to_csv("C:/Users/Jhack Alberth/Downloads/kagglecatsanddogs_5340/etiquetas_filtrado.csv", index=False)

In [11]:
# Usar el CSV filtrado para cargar el dataset
data = pd.read_csv("C:/Users/Jhack Alberth/Downloads/kagglecatsanddogs_5340/etiquetas_filtrado.csv")

# Continuar con el resto del código como se había planeado
# Crear el dataset desde el archivo CSV
ruta_dataset = tf.data.Dataset.from_tensor_slices((data["ruta"].values, data["etiqueta"].values))
dataset = ruta_dataset.map(lambda x, y: cargar_y_preprocesar_img(x, y), num_parallel_calls=tf.data.AUTOTUNE)

# Dividir en entrenamiento y validación
datos_entrenamiento = dataset.skip(int(len(dataset) * 0.15))
datos_validacion = dataset.take(int(len(dataset) * 0.15))

# Configuración de lotes para entrenamiento
datos_entrenamiento = datos_entrenamiento.batch(32).prefetch(tf.data.AUTOTUNE)
datos_validacion = datos_validacion.batch(32).prefetch(tf.data.AUTOTUNE)

In [12]:
# Definir el modelo de red neuronal convolucional (CNN)
modeloCNN = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(TAMANO_IMG, TAMANO_IMG, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

c:\Users\Jhack Alberth\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Compilar el modelo
modeloCNN.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [14]:
# Configuración de TensorBoard para visualizar el entrenamiento
tensorboardCNN = TensorBoard(log_dir='logs/cnn')

In [15]:
# Entrenar el modelo
modeloCNN.fit(
    datos_entrenamiento,
    validation_data=datos_validacion,
    epochs=10,  # Puedes ajustar el número de épocas según tus necesidades
    callbacks=[tensorboardCNN]
)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node DecodeJpeg defined at (most recent call last):
<stack traces unavailable>
Error in user-defined function passed to ParallelMapDatasetV2:11 transformation with iterator: Iterator::Root::Prefetch::BatchV2::FiniteSkip::ParallelMapV2: Trying to decode BMP format using a wrong op. Use `decode_bmp` or `decode_image` instead. Op used: DecodeJpeg
	 [[{{node DecodeJpeg}}]]
	 [[IteratorGetNext]] [Op:__inference_one_step_on_iterator_2142]